In [17]:
import argparse
import boto3
import evaluate
import importlib
import json
import lighteval
import os
import pathlib
import requests
import shutil
import sys
import tarfile
import time
import torch
import transformers
import uuid
import wandb

import awswrangler as wr
import numpy as np

from botocore.exceptions import ClientError
from datasets import load_dataset, DatasetDict, Dataset
from datetime import datetime, timezone
from IPython.display import display
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
from sagemaker import image_uris, utils as sm_utils
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from typing import List, Union, Optional


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config) 

_ = """
ml.g6.xlarge:   6     250GB
ml.g6.2xlarge:  3
ml.g6.4xlarge:  6
ml.g6.8xlarge:  6

google-bert/bert-base-uncased
distilbert/distilbert-base-uncased
microsoft/deberta-v3-base
FacebookAI/roberta-base
answerdotai/ModernBERT-base
allenai/scibert_scivocab_uncased
google/bigbird-roberta-base
allenai/longformer-base-4096
"""
script_dir

utils.py loaded: v0.2.12
config.py loaded: v0.1


PosixPath('/home/sagemaker-user/sagemaker_research_classification/src/05_tuning_basic')

In [18]:
RUNS = {}
MAX_RUNTIME = 2*24*60*60

In [4]:
# DONE
MODEL_NAME = 'bert-base-uncased'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-subfield-title-s1-0916224638'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-topic-title-s1-0917004738'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-subfield-abstract-s1-0917011627'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-topic-abstract-s1-0917012628'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'bert-subfield-fulltext-s1-0917013629'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'bert-topic-fulltext-s1-0917014630'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-subfield-title-s10-0917015631'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bert-topic-title-s10-0917020632'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-subfield-abstract-s10-0917021633'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-topic-abstract-s10-0917022634'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-subfield-fulltext-s10-0917023635'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-topic-fulltext-s10-0917024635'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-subfield-title-s100-0917025636'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bert-topic-title-s100-0917030637'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-subfield-abstract-s100-0917031638'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-topic-abstract-s100-0917032639'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-subfield-fulltext-s100-0917033640'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bert-topic-fulltext-s100-0917034641'},
]

In [ ]:
MODEL_NAME = 'bert-base-uncased'
for run in RUNS[MODEL_NAME]:
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(10*60)

Training job created: bert-subfield-abstract-s1-0917011627
Training job created: bert-topic-abstract-s1-0917012628
Training job created: bert-subfield-fulltext-s1-0917013629
Training job created: bert-topic-fulltext-s1-0917014630
Training job created: bert-topic-abstract-s100-0917032639
Training job created: bert-subfield-fulltext-s100-0917033640
Training job created: bert-topic-fulltext-s100-0917034641


In [11]:
# DONE
MODEL_NAME = 'distilbert-base-uncased'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-subfield-title-s1-0917084400'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-topic-title-s1-0917093942'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-subfield-abstract-s1-0917094943'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-topic-abstract-s1-0917095944'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'InProgress: distilbert-subfield-fulltext-s1-0917100945'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'distilbert-topic-fulltext-s1-0917101945'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-subfield-title-s10-0917102947'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'distilbert-topic-title-s10-0917103948'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'distilbert-subfield-abstract-s10-0917104949'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'distilbert-topic-abstract-s10-0917105949'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'distilbert-subfield-fulltext-s10-0917110950'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'distilbert-topic-fulltext-s10-0917111951'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'distilbert-subfield-title-s100-0917112952'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'distilbert-topic-title-s100-0917113953'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'distilbert-subfield-abstract-s100-0917114954'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'distilbert-topic-abstract-s100-0917115955'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'distilbert-subfield-fulltext-s100-0917120956'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'distilbert-topic-fulltext-s100-0917121957'},
]

In [9]:
MODEL_NAME = 'distilbert-base-uncased'
for run in RUNS[MODEL_NAME]:
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: distilbert-topic-title-s1-0917093942
Training job created: distilbert-subfield-abstract-s1-0917094943
Training job created: distilbert-topic-abstract-s1-0917095944
Training job created: distilbert-subfield-fulltext-s1-0917100945
Training job created: distilbert-topic-fulltext-s1-0917101945
Training job created: distilbert-subfield-title-s10-0917102947
Training job created: distilbert-topic-title-s10-0917103948
Training job created: distilbert-subfield-abstract-s10-0917104949
Training job created: distilbert-topic-abstract-s10-0917105949
Training job created: distilbert-subfield-fulltext-s10-0917110950
Training job created: distilbert-topic-fulltext-s10-0917111951
Training job created: distilbert-subfield-title-s100-0917112952
Training job created: distilbert-topic-title-s100-0917113953
Training job created: distilbert-subfield-abstract-s100-0917114954
Training job created: distilbert-topic-abstract-s100-0917115955
Training job created: distilbert-subfield-fulltext

In [5]:
# DONE
MODEL_NAME = 'microsoft/deberta-v3-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'deberta-subfield-title-s1-0917123950'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'deberta-topic-title-s1-0917131904'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'deberta-subfield-abstract-s1-0917132205'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'deberta-topic-abstract-s1-0917132506'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'deberta-subfield-fulltext-s1-0917132807'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'deberta-topic-fulltext-s1-0917133108'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'deberta-subfield-title-s10-0917133409'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'deberta-topic-title-s10-0917133710'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'deberta-subfield-abstract-s10-0917134010'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'deberta-topic-abstract-s10-0917134312'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'deberta-subfield-fulltext-s10-0917134613'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'deberta-topic-fulltext-s10-0917134914'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'deberta-subfield-title-s100-0917135215'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'deberta-topic-title-s100-0917135516'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'deberta-subfield-abstract-s100-0918144955'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'deberta-topic-abstract-s100-0918145257'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'deberta-subfield-fulltext-s100-0917140419'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'deberta-topic-fulltext-s100-0917140720'},
]

In [32]:
MODEL_NAME = 'microsoft/deberta-v3-base'
for run in RUNS[MODEL_NAME]:
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: deberta-subfield-abstract-s100-0918144955
Training job created: deberta-topic-abstract-s100-0918145257


In [6]:
# DONE
MODEL_NAME = 'roberta-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'roberta-subfield-title-s1-0917142512'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'roberta-topic-title-s1-0917153634'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'roberta-subfield-abstract-s1-0917153333'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'roberta-topic-abstract-s1-0917153032'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'roberta-subfield-fulltext-s1-0917152731'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'roberta-topic-fulltext-s1-0917152431'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'roberta-subfield-title-s10-0917152130'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'roberta-topic-title-s10-0917151829'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'roberta-subfield-abstract-s10-0917151528'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'roberta-topic-abstract-s10-0917151227'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'roberta-subfield-fulltext-s10-0917150926'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'roberta-topic-fulltext-s10-0917150625'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'roberta-subfield-title-s100-0917150324'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'roberta-topic-title-s100-0917150024'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'roberta-subfield-abstract-s100-0917145723'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'roberta-topic-abstract-s100-0917145422'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'roberta-subfield-fulltext-s100-0917145120-copy-09-17'}, # Testing manual restart on failure
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'roberta-topic-fulltext-s100-0917144819'},
]

In [32]:
MODEL_NAME = 'roberta-base'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: roberta-topic-fulltext-s100-0917144819
Training job created: roberta-subfield-fulltext-s100-0917145120
Training job created: roberta-topic-abstract-s100-0917145422
Training job created: roberta-subfield-abstract-s100-0917145723
Training job created: roberta-topic-title-s100-0917150024
Training job created: roberta-subfield-title-s100-0917150324
Training job created: roberta-topic-fulltext-s10-0917150625
Training job created: roberta-subfield-fulltext-s10-0917150926
Training job created: roberta-topic-abstract-s10-0917151227
Training job created: roberta-subfield-abstract-s10-0917151528
Training job created: roberta-topic-title-s10-0917151829
Training job created: roberta-subfield-title-s10-0917152130
Training job created: roberta-topic-fulltext-s1-0917152431
Training job created: roberta-subfield-fulltext-s1-0917152731
Training job created: roberta-topic-abstract-s1-0917153032
Training job created: roberta-subfield-abstract-s1-0917153333
Training job created: robe

In [7]:
# DONE
MODEL_NAME = 'answerdotai/ModernBERT-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'ModernBERT-subfield-title-s1-0917175205'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'ModernBERT-topic-title-s1-0917184208'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'ModernBERT-subfield-abstract-s1-0917183907'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'ModernBERT-topic-abstract-s1-0917183606'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'ModernBERT-subfield-fulltext-s1-0917183306'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'ModernBERT-topic-fulltext-s1-0917183004'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'ModernBERT-subfield-title-s10-0917182703'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'ModernBERT-topic-title-s10-0917182403'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'ModernBERT-subfield-abstract-s10-0917182102'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'ModernBERT-topic-abstract-s10-0917181801'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'ModernBERT-subfield-fulltext-s10-0917181500'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'ModernBERT-topic-fulltext-s10-0917181159'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'ModernBERT-subfield-title-s100-0917180858'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'ModernBERT-topic-title-s100-0917180557'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'ModernBERT-subfield-abstract-s100-0918004625'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'ModernBERT-topic-abstract-s100-0918004324'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'ModernBERT-subfield-fulltext-s100-0918144655'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'ModernBERT-topic-fulltext-s100-0918144354'},
]

In [30]:
MODEL_NAME = 'answerdotai/ModernBERT-base'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME,
            # To prevent "RuntimeError: Detected that you are using FX to symbolically trace a dynamo-optimized function. This is not supported at the moment."
            # Source: https://huggingface.co/answerdotai/ModernBERT-base/discussions/14
            SCP_REFERENCE_COMPILE=False,
            SCP_ATTN_IMPLEMENTATION='eager'
        )
        time.sleep(3*60)

Training job created: ModernBERT-topic-fulltext-s100-0918144354
Training job created: ModernBERT-subfield-fulltext-s100-0918144655


In [8]:
# DONE
MODEL_NAME = 'allenai/scibert_scivocab_uncased'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'scibert-subfield-title-s1-0917143225'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'scibert-topic-title-s1-0917222652'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'scibert-subfield-abstract-s1-0917222351'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'scibert-topic-abstract-s1-0917222050'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'scibert-subfield-fulltext-s1-0917221749'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'scibert-topic-fulltext-s1-0917221448'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'scibert-subfield-title-s10-0917221147'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'scibert-topic-title-s10-0917220846'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'scibert-subfield-abstract-s10-0917220545'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'scibert-topic-abstract-s10-0917220245'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'scibert-subfield-fulltext-s10-0917215944'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'scibert-topic-fulltext-s10-0917215643'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'scibert-subfield-title-s100-0917215342'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'scibert-topic-title-s100-0917215041'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'scibert-subfield-abstract-s100-0917214740'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'scibert-topic-abstract-s100-0917214440'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'scibert-subfield-fulltext-s100-0917214139'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'scibert-topic-fulltext-s100-0917213838'},
]

In [48]:
MODEL_NAME = 'allenai/scibert_scivocab_uncased'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: scibert-topic-fulltext-s100-0917213838
Training job created: scibert-subfield-fulltext-s100-0917214139
Training job created: scibert-topic-abstract-s100-0917214440
Training job created: scibert-subfield-abstract-s100-0917214740
Training job created: scibert-topic-title-s100-0917215041
Training job created: scibert-subfield-title-s100-0917215342
Training job created: scibert-topic-fulltext-s10-0917215643
Training job created: scibert-subfield-fulltext-s10-0917215944
Training job created: scibert-topic-abstract-s10-0917220245
Training job created: scibert-subfield-abstract-s10-0917220545
Training job created: scibert-topic-title-s10-0917220846
Training job created: scibert-subfield-title-s10-0917221147
Training job created: scibert-topic-fulltext-s1-0917221448
Training job created: scibert-subfield-fulltext-s1-0917221749
Training job created: scibert-topic-abstract-s1-0917222050
Training job created: scibert-subfield-abstract-s1-0917222351
Training job created: scib

In [9]:
# DONE
MODEL_NAME = 'google/bigbird-roberta-base'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bigbird-subfield-title-s1-0917143526'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bigbird-topic-title-s1-0918024259'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bigbird-subfield-abstract-s1-0918023958'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bigbird-topic-abstract-s1-0918023658'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'bigbird-subfield-fulltext-s1-0918023356'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'bigbird-topic-fulltext-s1-0918023055'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bigbird-subfield-title-s10-0918022755'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'bigbird-topic-title-s10-0918022454'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bigbird-subfield-abstract-s10-0918022153'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bigbird-topic-abstract-s10-0918021852'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bigbird-subfield-fulltext-s10-0918021552'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bigbird-topic-fulltext-s10-0918021250'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bigbird-subfield-title-s100-0918020949'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'bigbird-topic-title-s100-0918020649'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bigbird-subfield-abstract-s100-0918020348'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bigbird-topic-abstract-s100-0918020047'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bigbird-subfield-fulltext-s100-0918015746'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'bigbird-topic-fulltext-s100-0918015446'},
]

In [53]:
MODEL_NAME = 'google/bigbird-roberta-base'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: bigbird-topic-fulltext-s100-0918015446
Training job created: bigbird-subfield-fulltext-s100-0918015746
Training job created: bigbird-topic-abstract-s100-0918020047
Training job created: bigbird-subfield-abstract-s100-0918020348
Training job created: bigbird-topic-title-s100-0918020649
Training job created: bigbird-subfield-title-s100-0918020949
Training job created: bigbird-topic-fulltext-s10-0918021250
Training job created: bigbird-subfield-fulltext-s10-0918021552
Training job created: bigbird-topic-abstract-s10-0918021852
Training job created: bigbird-subfield-abstract-s10-0918022153
Training job created: bigbird-topic-title-s10-0918022454
Training job created: bigbird-subfield-title-s10-0918022755
Training job created: bigbird-topic-fulltext-s1-0918023055
Training job created: bigbird-subfield-fulltext-s1-0918023356
Training job created: bigbird-topic-abstract-s1-0918023658
Training job created: bigbird-subfield-abstract-s1-0918023958
Training job created: bigb

In [12]:
MODEL_NAME = 'allenai/longformer-base-4096'
RUNS[MODEL_NAME] = [
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'longformer-subfield-title-s1-0917143827'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'longformer-topic-title-s1-0918110303'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'longformer-subfield-abstract-s1-0918110003'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'longformer-topic-abstract-s1-0918105701'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'longformer-subfield-fulltext-s1-0918105400'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '1', 'INSTANCE_TYPE': 'ml.g6.2xlarge',
     'done': 'longformer-topic-fulltext-s1-0918105100'},

    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'longformer-subfield-title-s10-0918104759'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.xlarge',
     'done': 'longformer-topic-title-s10-0918104458'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'longformer-subfield-abstract-s10-0918104157'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'longformer-topic-abstract-s10-0918103856'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'longformer-subfield-fulltext-s10-0918103555'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '10', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'longformer-topic-fulltext-s10-0918103254'},
    
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'longformer-subfield-title-s100-0918102953'},
    {'TEXT_KEY': 'title', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.4xlarge',
     'done': 'longformer-topic-title-s100-0918102652'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'longformer-subfield-abstract-s100-0918102352'},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'longformer-topic-abstract-s100-0918102051'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'longformer-subfield-fulltext-s100-0918101749'},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': 'longformer-topic-fulltext-s100-0918101448'},
]

In [4]:
MODEL_NAME = 'allenai/longformer-base-4096'
for run in reversed(RUNS[MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME
        )
        time.sleep(3*60)

Training job created: longformer-topic-fulltext-s100-0918101448
Training job created: longformer-subfield-fulltext-s100-0918101749
Training job created: longformer-topic-abstract-s100-0918102051
Training job created: longformer-subfield-abstract-s100-0918102352
Training job created: longformer-topic-title-s100-0918102652
Training job created: longformer-subfield-title-s100-0918102953
Training job created: longformer-topic-fulltext-s10-0918103254
Training job created: longformer-subfield-fulltext-s10-0918103555
Training job created: longformer-topic-abstract-s10-0918103856
Training job created: longformer-subfield-abstract-s10-0918104157
Training job created: longformer-topic-title-s10-0918104458
Training job created: longformer-subfield-title-s10-0918104759
Training job created: longformer-topic-fulltext-s1-0918105100
Training job created: longformer-subfield-fulltext-s1-0918105400
Training job created: longformer-topic-abstract-s1-0918105701
Training job created: longformer-subfield-a

In [19]:
MODEL_NAME = 'allenai/scibert_scivocab_uncased'
USE_SEMIBALANCED = True
RUNS['semibalanced_'+MODEL_NAME] = [
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'abstract', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'subfield', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
    {'TEXT_KEY': 'fulltext', 'LABEL_TYPE': 'topic', 'SAMPLE': '100', 'INSTANCE_TYPE': 'ml.g6.8xlarge',
     'done': False},
]

In [20]:
MODEL_NAME = 'allenai/scibert_scivocab_uncased'
for run in reversed(RUNS['semibalanced_'+MODEL_NAME]):
    if 'done' not in run or not run['done']:
        JOB_NAME = utils.create_supervised_multiclass_classification_training_job(
            SCRIPT_FILEPATH=script_dir,
            MODEL_NAME=MODEL_NAME,
            INSTANCE_TYPE=run['INSTANCE_TYPE'],
            TEXT_KEY=run['TEXT_KEY'],
            LABEL_TYPE=run['LABEL_TYPE'],
            SAMPLE=run['SAMPLE'],
            MAX_RUNTIME_S=MAX_RUNTIME,
            USE_SEMIBALANCED=USE_SEMIBALANCED
        )
        time.sleep(3*60)

Training job created: semibalanced-scibert-topic-fulltext-s100-0920163933
Training job created: semibalanced-scibert-subfield-fulltext-s100-0920164234
Training job created: semibalanced-scibert-topic-abstract-s100-0920164535
Training job created: semibalanced-scibert-subfield-abstract-s100-0920164836


In [13]:
for model_name, model_runs in RUNS.items():
    for model_run in model_runs:
        if 'done' not in model_run or not model_run['done']:
            # longformer-subfield-title-s1-0917143827
            model_short_name = model_name.replace('_','-').split("/")[-1].split("-")[0]
            job_name_prefix = f'{model_short_name}-{model_run["LABEL_TYPE"]}-{model_run["TEXT_KEY"]}-s{model_run["SAMPLE"]}-'
            print(job_name_prefix)